In [20]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append("..")
from utils import *
from models import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [21]:
train_tape = "Z:/rppg/pure_datatape_160x36x36_train.h5"
valid_tape = "Z:/rppg/pure_datatape_160x36x36_valid.h5"

train =  load_datatape(train_tape, use_normalized_bvp=True)
valid = load_datatape(valid_tape, use_normalized_bvp=False)

def to_tf_deep_phys(datatape, dtype=tf.float16):
    def _():
        for i, j in datatape:
            j = tf.concat([j[1:]-j[:-1], [0]], axis=0)
            #yield i, (j-tf.reduce_mean(j, axis=0))/(tf.math.reduce_std(j, axis=0)+1e-6)
            yield i, j
    return tf.data.Dataset.from_generator(lambda :_(), output_types=(dtype, dtype), output_shapes=(datatape.shape, datatape.shape[:1]))

#train, valid = to_tf_deep_phys(train).cache(f'{tmp}/deepphys'), to_tf_deep_phys(valid).cache(f'{tmp}/deepphys')
train, valid = to_tf_deep_phys(train), to_tf_deep_phys(valid)

In [22]:
deepphys = DeepPhys_end_to_end()
deepphys.compile(optimizer='Nadam', loss='mse')
deepphys.build(input_shape=(None, 36, 36, 3))
print(f'Flops per frame:{get_flops(deepphys, input_sig=[tf.TensorSpec([160, 36, 36, 3])])/160:.0f}')
deepphys.summary()

Flops per frame:52158718
Model: "deep_phys_end_to_end_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
model_2 (Functional)         (None, 1)                 532899    
Total params: 532,899
Trainable params: 532,899
Non-trainable params: 0
_________________________________________________________________


In [23]:
deepphys.fit(train.shuffle(2000), validation_data=valid, epochs=10, callbacks=[keras.callbacks.ModelCheckpoint('../weights/DeepPhys_PURE.h5', monitor='val_loss', verbose=1, save_best_only=True, mode='min', save_weights_only=True)])
deepphys.load_weights('../weights/DeepPhys_PURE.h5')

Epoch 1/10
780/780 [==============================] - 21s 22ms/step - loss: 0.1680 - val_loss: 0.1389

Epoch 00001: val_loss improved from inf to 0.13888, saving model to ../weights\DeepPhys_PURE.h5
Epoch 2/10
780/780 [==============================] - 20s 22ms/step - loss: 0.1850 - val_loss: 0.1376

Epoch 00002: val_loss improved from 0.13888 to 0.13756, saving model to ../weights\DeepPhys_PURE.h5
Epoch 3/10
780/780 [==============================] - 20s 22ms/step - loss: 0.1589 - val_loss: 0.1323

Epoch 00003: val_loss improved from 0.13756 to 0.13226, saving model to ../weights\DeepPhys_PURE.h5
Epoch 4/10
780/780 [==============================] - 20s 22ms/step - loss: 0.1562 - val_loss: 0.1269

Epoch 00004: val_loss improved from 0.13226 to 0.12693, saving model to ../weights\DeepPhys_PURE.h5
Epoch 5/10
780/780 [==============================] - 32s 36ms/step - loss: 0.1537 - val_loss: 0.1248

Epoch 00005: val_loss improved from 0.12693 to 0.12476, saving model to ../weights\DeepPh

In [24]:
def cumsum(result, overwrite=True):
    with h5py.File(result, 'a') as f:
        for i, j in f.items():
            if 'diff' not in j.keys():
                j.create_dataset('diff', data=j['predict'])
                j['predict'][...] = detrend(np.cumsum(j['diff']))
            elif overwrite:
                j['predict'][...] = detrend(np.cumsum(j['diff']))

In [25]:
eval_on_dataset(test_set_UBFC_rPPG2, deepphys, 160, (36, 36), step=1, batch=0, save='../results/DeepPhys_PURE_UBFC.h5', sample=cv2.INTER_CUBIC)
cumsum('../results/DeepPhys_PURE_UBFC.h5')
get_metrics('../results/DeepPhys_PURE_UBFC.h5')

100%|██████████| 42/42 [00:20<00:00,  2.01it/s]


{'Sliding window': {'MAE': 4.714, 'RMSE': 11.605, 'R': 0.82481},
 'Whole video': {'MAE': 3.124, 'RMSE': 8.774, 'R': 0.89676}}